In [34]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [101]:
import warnings
warnings.filterwarnings('ignore')

In [93]:
df = pd.read_csv('datasets/train_set.csv')

test_df = pd.read_csv('datasets/test_set.csv')

In [94]:
X = df.loc[:, 'month':]
y = df['cpi_pct']


X_test = test_df.loc[:, X.columns]

In [99]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [ ]:
# Shuffling lead to very high lambda values wich basicly meant moth models were just predicting mean (y intercept)
# X = X.sample(frac=1)

In [129]:
model = LinearRegression()

model.fit(X, y)

pred = model.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv('predictions/ols_pred.csv') 

In [130]:
model = Lasso(max_iter=2000)

params = {
    'alpha': np.logspace(-8, -0.5, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv('predictions/lassoCV_pred.csv')

Lasso(alpha=0.00014677992676220705, max_iter=2000)


In [127]:
model = Ridge()

params = {
    'alpha': np.linspace(1, 10000, 100).tolist()
}

cv = GridSearchCV(model, params, cv=10)
cv.fit(X, y)

print(cv.best_estimator_)

pred = cv.predict(X_test)

pd.DataFrame(pred, index=X_test.index, columns=['pred']).to_csv('predictions/ridgeCV_pred.csv')

Ridge(alpha=607.0)
